## Envinroment preparation

First line if **magic** enabling matplotlib inline plots 

In [1]:
%matplotlib inline

Then we have round of inports:
* **pandas** is our main data storage module
* **glob** and **os** are used for filename manipulations

In [2]:
import pandas as pd
import glob
import os

## Files index and identification

In [3]:
filenames = [ os.path.splitext(wholeFilename)[0] for wholeFilename in 
             [ os.path.basename(wholePath) for wholePath in glob.glob("../input/2*.xlsx") ] ]

In [4]:
dataFiles = pd.DataFrame({"filename": filenames})
dataFiles["year"], dataFiles["pollutant"], dataFiles["resolution"] = dataFiles["filename"].str.split('_', 2).str

In [5]:
dataFiles.head()

,filename,year,pollutant,resolution
0,2010_Pb(PM10)_24g,2010,Pb(PM10),24g
1,2007_As(PM10)_24g,2007,As(PM10),24g
2,2005_PM2.5_24g,2005,PM2.5,24g
3,2009_PM2.5_1g,2009,PM2.5,1g
4,2011_NH4+(PM2.5)_24g,2011,NH4+(PM2.5),24g


In [6]:
dataFiles["year"].value_counts()

2012    35
2011    35
2014    35
2013    34
2015    29
2010    25
2009    24
2008    24
2007    18
2006    18
2005    17
2004    17
2003    17
2002    14
2001    11
2000     6
Name: year, dtype: int64

In [7]:
dataFiles["pollutant"].value_counts()

NO2            32
SO2            31
PM10           29
C6H6           28
PM2.5          22
O3             16
NOx            16
Cd(PM10)       15
Ni(PM10)       15
BaP(PM10)      15
Pb(PM10)       14
As(PM10)       14
CO             13
IP(PM10)        8
BjF(PM10)       8
BbF(PM10)       8
BkF(PM10)       8
BaA(PM10)       8
DBahA(PM10)     7
formaldehyd     5
Na+(PM2.5)      4
NH4+(PM2.5)     4
NO3-(PM2.5)     4
Ca2+(PM2.5)     4
OC(PM2.5)       4
Mg2+(PM2.5)     4
Cl              4
EC(PM2.5)       4
SO42            4
K+(PM2.5)       4
PM25            2
Hg(TGM)         2
DBah(PM10)      1
depozycja       1
Jony            1
Name: pollutant, dtype: int64

In [8]:
dataFiles["resolution"].value_counts()

24g            236
1g             113
(PM2.5)-24g      4
(PM2.5)_24g      4
w_PM25_24g       1
Name: resolution, dtype: int64

## Fixing data files identification

In [9]:
dataFiles[dataFiles["resolution"] == "(PM2.5)-24g"]

,filename,year,pollutant,resolution
115,2013_Cl_(PM2.5)-24g,2013,Cl,(PM2.5)-24g
192,2014_Cl_(PM2.5)-24g,2014,Cl,(PM2.5)-24g
300,2011_Cl_(PM2.5)-24g,2011,Cl,(PM2.5)-24g
301,2012_Cl_(PM2.5)-24g,2012,Cl,(PM2.5)-24g


In [10]:
dataFiles.ix[dataFiles["resolution"] == "(PM2.5)-24g", 'pollutant'] = "Cl_(PM2.5)"
dataFiles.ix[dataFiles["resolution"] == "(PM2.5)-24g", 'resolution'] = "24g"

In [11]:
dataFiles[dataFiles["pollutant"] == "Cl_(PM2.5)"]

,filename,year,pollutant,resolution
115,2013_Cl_(PM2.5)-24g,2013,Cl_(PM2.5),24g
192,2014_Cl_(PM2.5)-24g,2014,Cl_(PM2.5),24g
300,2011_Cl_(PM2.5)-24g,2011,Cl_(PM2.5),24g
301,2012_Cl_(PM2.5)-24g,2012,Cl_(PM2.5),24g


In [12]:
dataFiles[dataFiles["resolution"] == "(PM2.5)_24g"]

,filename,year,pollutant,resolution
39,2014_SO42_(PM2.5)_24g,2014,SO42,(PM2.5)_24g
186,2013_SO42_(PM2.5)_24g,2013,SO42,(PM2.5)_24g
191,2011_SO42_(PM2.5)_24g,2011,SO42,(PM2.5)_24g
292,2012_SO42_(PM2.5)_24g,2012,SO42,(PM2.5)_24g


In [13]:
dataFiles.ix[dataFiles["resolution"] == "(PM2.5)_24g", 'pollutant'] = "SO42_(PM2.5)"
dataFiles.ix[dataFiles["resolution"] == "(PM2.5)_24g", 'resolution'] = "24g"

In [14]:
dataFiles[dataFiles["pollutant"] == "SO42_(PM2.5)"]

,filename,year,pollutant,resolution
39,2014_SO42_(PM2.5)_24g,2014,SO42_(PM2.5),24g
186,2013_SO42_(PM2.5)_24g,2013,SO42_(PM2.5),24g
191,2011_SO42_(PM2.5)_24g,2011,SO42_(PM2.5),24g
292,2012_SO42_(PM2.5)_24g,2012,SO42_(PM2.5),24g


In [15]:
dataFiles[dataFiles["resolution"] == "w_PM25_24g"]

,filename,year,pollutant,resolution
129,2015_Jony_w_PM25_24g,2015,Jony,w_PM25_24g


In [16]:
dataFiles.ix[dataFiles["resolution"] == "w_PM25_24g", 'pollutant'] = "Jony_w_PM25"
dataFiles.ix[dataFiles["resolution"] == "w_PM25_24g", 'resolution'] = "24g"

In [17]:
dataFiles[dataFiles["pollutant"] == "Jony_w_PM25"]

,filename,year,pollutant,resolution
129,2015_Jony_w_PM25_24g,2015,Jony_w_PM25,24g


Now **resolution** column should be correct:

In [18]:
dataFiles["resolution"].value_counts()

24g    245
1g     113
Name: resolution, dtype: int64

In [19]:
dataFiles.describe()

,filename,year,pollutant,resolution
count,359,359,359,358
unique,359,16,35,2
top,2012_NO2_24g,2012,NO2,24g
freq,1,35,32,245


There is still one empty cell in **resolution** column. Lets identify it:

In [20]:
dataFiles[dataFiles["resolution"].isnull()]

,filename,year,pollutant,resolution
312,2015_depozycja,2015,depozycja,NaN


After manually examinign **2015_depozycja** file I found that it cointains new type of data, which will be useless in planned analysis. I decided to remove it from working memory. 

In [21]:
dataFiles.drop(dataFiles[dataFiles["filename"] == "2015_depozycja"].index, inplace=True)

In [22]:
dataFiles.describe()

,filename,year,pollutant,resolution
count,358,358,358,358
unique,358,16,34,2
top,2012_NO2_24g,2012,NO2,24g
freq,1,35,32,245


## Looking for worst measuring station for each pollutant in 2015

In [23]:
importantPollutants = ["PM10", "PM25", "O3", "NO2", "SO2", "C6H6", "CO"]
pollutants2015 = dataFiles[(dataFiles["year"] == "2015") & (dataFiles["resolution"] == "1g") & 
                           (dataFiles["pollutant"].isin(importantPollutants))]

In [24]:
pollutants2015

,filename,year,pollutant,resolution
14,2015_NO2_1g,2015,NO2,1g
106,2015_CO_1g,2015,CO,1g
138,2015_SO2_1g,2015,SO2,1g
141,2015_O3_1g,2015,O3,1g
207,2015_C6H6_1g,2015,C6H6,1g
310,2015_PM10_1g,2015,PM10,1g
326,2015_PM25_1g,2015,PM25,1g


In [25]:
from tqdm import tqdm

In [26]:
from collections import Counter

In [27]:
#worstStation = {}
#for index, dataRow in tqdm(pollutants2015.iterrows(), total=len(pollutants2015.index)):
#    dataFromFile = pd.read_excel("../input/" + dataRow["filename"] + ".xlsx", skiprows=[1,2])
#    dataFromFile = dataFromFile.rename(columns={"Kod stacji":"Godzina"})
#    dataFromFile = dataFromFile.set_index("Godzina")
#    worstStation[dataRow["pollutant"]] = dataFromFile.max().sort_values(ascending = False).index[0]

## Building one big data frame

In [28]:
data1 = pd.read_excel("../input/2015_PM10_1g.xlsx", skiprows=[1,2])
data2 = pd.read_excel("../input/2015_PM25_1g.xlsx", skiprows=[1,2])

In [29]:
data1 = data1.rename(columns={"Kod stacji":"Hour"})
data2 = data2.rename(columns={"Kod stacji":"Hour"})

In [30]:
rng = pd.date_range(start = '2015-01-01 01:00:00', end = '2016-01-01 00:00:00', freq='H')
data1["Hour"] = rng
data2["Hour"] = rng

In [31]:
data1 = data1.set_index("Hour")
data2 = data2.set_index("Hour")

In [32]:
data1 = data1.stack()
data2 = data2.stack()

In [33]:
data1 = pd.DataFrame(data1, columns=["PM10"])
data2 = pd.DataFrame(data2, columns=["PM25"])

In [34]:
data1.index.set_names(['Hour', 'Station'], inplace=True)
data2.index.set_names(['Hour', 'Station'], inplace=True)

In [35]:
dataMerged = pd.concat([data1, data2], axis=1)

In [36]:
dataMerged[dataMerged["PM25"].notnull() & dataMerged["PM10"].notnull()]

PM10        PM25
Hour                Station                             
2015-01-01 01:00:00 KpBydPlPozna  121.819179   29.200000
                    KpBydWarszaw  109.400000   95.500000
                    KpToruDziewu  111.377000   27.352000
                    LbLubObywate   73.823000   71.101200
                    LdZgieMielcz   71.576782   58.690689
                    MpKrakAlKras  108.000000   73.000000
                    MpKrakBujaka  124.000000   88.000000
                    MpKrakBulwar   75.000000   58.000000
                    MzPlocMiReja   56.270780   47.075769
                    MzRadTochter   64.063587   55.465511
                    MzSiedKonars   47.401700   43.655099
                    MzWarAlNiepo   84.235700   71.820400
                    MzWarKondrat   87.710000   78.085000
                    MzWarWokalna   80.638763   68.982316
                    MzZyraRoosev   88.857545   86.182337
                    OpKKozBSmial   40.000000   43.000000
                    PkMielSolski   53.411600   55.797000
                    PkPrzemGrunw   19.744500   18.902700
                    SkKielJagiel   55.400000   56.600000
                    SkMalo11List   44.600000   39.600000
                    SkPolaRuszcz   70.700000   72.400000
                    SlGliwicMewy  109.534000   77.828000
                    SlKatoKossut   56.936000   51.389000
                    SlZlotPotLes   53.023400   49.946400
                    WmOlsPuszkin   78.692543   44.060833
                    WpKaliSawick   28.300000   25.500000
                    WpPoznPolank   35.729000   24.476800
2015-01-01 02:00:00 DsJelGorOgin  307.006000  262.566000
                    KpBydPlPozna   38.545058   27.100000
                    KpBydWarszaw   97.400000   86.100000
...                                      ...         ...
2016-01-01 00:00:00 KpBydWarszaw   73.000000   62.200000
                    LbLubObywate  234.972500  205.570400
                    LdLodzCzerni   50.000000   50.000000
                    LdLodzGdansk   56.300000   59.000000
                    LdZgieMielcz   36.000000   30.000000
                    LuZielKrotka   58.155400   57.347900
                    MpKrakAlKras  254.931000  211.441000
                    MpKrakBujaka  414.837000  281.380000
                    MpKrakBulwar  270.034000  215.873000
                    MzPlocMiReja   55.775998   45.211383
                    MzRadTochter  117.623671  101.842135
                    MzSiedKonars  110.545700  110.090915
                    MzWarAlNiepo   68.625100   57.096400
                    MzWarKondrat   73.710000   68.335000
                    MzWarWokalna   72.227480   59.310809
                    MzZyraRoosev   84.344146   79.059829
                    OpKKozBSmial   18.940600   18.400000
                    PkMielSolski  138.849000  120.034000
                    PkPrzemGrunw  110.149000  109.851000
                    PmGdaLeczk08   42.185700   39.640300
                    SkKielJagiel  103.000000   87.100000
                    SkMalo11List   51.800000   41.600000
                    SkPolaRuszcz  138.370000  113.200000
                    SlGliwicMewy  113.084000   92.868700
                    SlKatoKossut  157.855000  133.811000
                    SlZlotPotLes   53.907800   51.620700
                    WmOlsPuszkin   47.043335   40.265392
                    WpKaliSawick   95.235000   87.123600
                    WpPoznPolank   68.198500   55.935100
                    ZpSzczAndr01   83.180500   76.853900

[277856 rows x 2 columns]

## Working with stations 

In [ ]:
stations = pd.read_excel("../input/Metadane_wer20160914.xlsx")

In [ ]:
nazywStacji = set(dane.columns.values)

In [ ]:
stacje = stacje.set_index("Nr")

In [ ]:
stacje[(stacje["Stary Kod stacji"]).isin(nazywStacji) | (stacje["Kod stacji"]).isin(nazywStacji)]

In [ ]:
interesujaceStacje = stacje[(stacje["Stary Kod stacji"]).isin(nazywStacji) | (stacje["Kod stacji"]).isin(nazywStacji)]

In [ ]:
interesujaceStacje

In [ ]:
interesujaceStacje.shape

In [ ]:
interesujaceStacje[[u'WGS84 \u03bb E',u'WGS84 \u03c6 N']]

In [ ]:
wspolrzedne = interesujaceStacje[[u'WGS84 \u03bb E',u'WGS84 \u03c6 N']].values

In [ ]:
wspolrzedne[:,1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(wspolrzedne[:,0], wspolrzedne[:,1])

In [ ]:
import folium
map_osm = folium.Map(location=[52.069167, 19.480556], zoom_start=6)

In [ ]:
map_osm

In [ ]:
interesujaceStacje.index

In [ ]:
for index, row in interesujaceStacje.iterrows():
    print row['Nazwa stacji']
    folium.Marker([row[u'WGS84 \u03c6 N'], row[u'WGS84 \u03bb E']], popup=row['Nazwa stacji']).add_to(map_osm)

In [ ]:
map_osm

In [ ]:
jeden_dzien = dane[dane.index == "2000-06-12 08:00:00"]

In [ ]:
do_interpolacji = pd.melt(jeden_dzien)

In [ ]:
do_interpolacji.rename(columns={"variable":"Stary Kod stacji"}, inplace=True)

In [ ]:
final = do_interpolacji.merge(interesujaceStacje[[u'WGS84 \u03bb E',u'WGS84 \u03c6 N', "Stary Kod stacji"]])

In [ ]:
x = final[u'WGS84 \u03bb E'].values
y = final[u'WGS84 \u03c6 N'].values
z = final[u'value'].values

In [ ]:
x

In [ ]:
import numpy as np
from scipy.interpolate import griddata

In [ ]:
xi = np.linspace(x.min(),x.max(),100)
yi = np.linspace(y.min(),y.max(),200)

xi = np.append(xi,x)
xi.sort()
yi = np.append(yi,y)
yi.sort()

zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')

In [ ]:
(x,y), z

In [ ]:
zi

In [ ]:
# contour the gridded data, plotting dots at the randomly spaced data points.
CS = plt.contour(xi,yi,zi)
CS = plt.contourf(xi,yi,zi)
plt.colorbar() # draw colorbar
# plot data points.
plt.scatter(x,y,marker='o',c='b',s=5)
plt.show()

In [ ]:
from folium import plugins

In [ ]:
nxpoints = (x.max() - x.min()) / .001
nypoints = (y.max() - y.min()) / .001

In [ ]:
xi = np.linspace(x.min(),x.max(),int(nxpoints))
yi = np.linspace(y.min(),y.max(),int(nypoints))

In [ ]:
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')

In [ ]:
print(xi.shape)
print(yi.shape)
np.isnan(zi[4502,5000])

In [ ]:
from tqdm import tqdm

In [ ]:
xlist = []
ylist = []
zlist = []

In [ ]:
for xelement in tqdm(range(xi.shape[0])):
    for yelement in range(yi.shape[0]):
        if np.isnan(zi[yelement,xelement]):
            pass
        else:
            #tmpData = pd.DataFrame()
            #tmpData["x"] = xi[xelement]
            xlist.append(xi[xelement])
            ylist.append(yi[yelement])
            zlist.append(zi[yelement,xelement])
            #tmpData["y"] = yi[yelement]
            #tmpData["z"] = zi[yelement,xelement]
            #dataForHeatmap.append(tmpData, ignore_index=True)

In [ ]:
dataForHeatmap = pd.DataFrame({"x":xlist, "y":ylist, "z":zlist})

In [ ]:
dataForHeatmap

In [ ]:
#plugins.HeatMap(zip(ylist, xlist, zlist)).add_to(map_osm)

In [ ]:
#map_osm

In [ ]:
file

In [ ]:
[ basename(wholeFilename) for wholeFilename in glob.glob("../input/2*.xlsx") ]


In [ ]:
["asdfasdf", "asdfa", "asdf"]

In [ ]:
list.